In [12]:
# import pandas as pd
# import matplotlib.pyplot as plt
    
# train_data = pd.read_csv("./savefolderpytorch/train.csv")
# val_data = pd.read_csv("./savefolderpytorch/val.csv")


# train_data_epoch = train_data.groupby('epoch')['loss'].mean()
# val_data_epoch = val_data.groupby('epoch')['loss'].mean()

# plt.plot(train_data_epoch.index, train_data_epoch.values)
# plt.plot(val_data_epoch.index, val_data_epoch.values)
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.legend(['Train', 'Validation'])
# plt.show()

In [13]:
import torch
import os
import numpy as np
import pandas as pd
from warnings import simplefilter

import matplotlib.pyplot as plt

MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

from DataLoader import *
from Trainer import *
from Model import *
from RESNET import *

In [14]:


model_path = './savefolderpytorch/model3_2.pt'

# instantiate your model
model = CNNModel3() 

# load your model. Here we're loading on CPU since we're not going to do 
# large amounts of inference
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'))) 

# put it in evaluation mode for inference
model.eval()

model.conv3

c:\Users\20192584\Anaconda3\envs\AI000\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [15]:
# defines two global scope variables to store our gradients and activations
gradients = None
activations = None

def backward_hook(module, grad_input, grad_output):
  global gradients # refers to the variable in the global scope
  print('Backward hook running...')
  gradients = grad_output
  # In this case, we expect it to be torch.Size([batch size, 1024, 8, 8])
  print(f'Gradients size: {gradients[0].size()}') 
  # We need the 0 index because the tensor containing the gradients comes
  # inside a one element tuple.

def forward_hook(module, args, output):
  global activations # refers to the variable in the global scope
  print('Forward hook running...')
  activations = output
  # In this case, we expect it to be torch.Size([batch size, 1024, 8, 8])
  print(f'Activations size: {activations.size()}')

backward_hook = model.conv1.register_full_backward_hook(backward_hook)
forward_hook = model.conv1.register_forward_hook(forward_hook)


In [16]:
from PIL import Image

img_path = '../Solid_droplet/Data/67.0_32.0-1.0-20.0-13-0.png'
image = Image.open(img_path).convert('RGB')

from torchvision import transforms
from torchvision.transforms import ToTensor

image_size = 256
transform = transforms.Compose([

                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

img_tensor = transform(image) # stores the tensor that represents the image

In [17]:
model(img_tensor.unsqueeze(0)).backward()

Forward hook running...
Activations size: torch.Size([1, 8, 1076, 1916])
Backward hook running...
Gradients size: torch.Size([1, 8, 1076, 1916])


In [18]:
# pool the gradients across the channels
pooled_gradients = torch.mean(gradients[0], dim=[0, 2, 3])



In [19]:
import torch.nn.functional as F
from torchvision import utils
from torchvision.transforms import ToPILImage

# weight the channels by corresponding gradients
for i in range(activations.size()[1]):
    activations[:, i, :, :] *= pooled_gradients[i]

# average the channels of the activations
heatmap = torch.mean(activations, dim=1).squeeze()

# relu on top of the heatmap
heatmap = F.relu(heatmap)

# normalize the heatmap
heatmap /= torch.max(heatmap)

# # draw the heatmap
# plt.matshow(heatmap.detach())

# Convert the heatmap to a PIL image
heatmap_normalized = torch.clamp(heatmap, 0, 1)  # Ensure values are between 0 and 1
heatmap_pil = ToPILImage()(heatmap_normalized)

# Save the heatmap as an image
heatmap_pil.save('heatmap5.png')

In [20]:


# from torchvision.transforms.functional import to_pil_image
# from matplotlib import colormaps
# import numpy as np
# import PIL

# # Create a figure and plot the first image
# fig, ax = plt.subplots()
# ax.axis('off') # removes the axis markers

# # First plot the original image
# ax.imshow(to_pil_image(img_tensor, mode='RGB'))

# # Resize the heatmap to the same size as the input image and defines
# # a resample algorithm for increasing image resolution
# # we need heatmap.detach() because it can't be converted to numpy array while
# # requiring gradients
# overlay = to_pil_image(heatmap.detach(), mode='F').resize((256,256), resample=PIL.Image.BICUBIC)

# # Apply any colormap you want
# cmap = colormaps['jet']
# overlay = (255 * cmap(np.asarray(overlay) ** 2)[:, :, :3]).astype(np.uint8)

# # Plot the heatmap on the same axes, 
# # but with alpha < 1 (this defines the transparency of the heatmap)
# ax.imshow(overlay, alpha=0.4, interpolation='nearest', extent=extent)

# # Show the plot
# plt.show()